In [ ]:
%%capture
!pip install sentence_transformers
!pip install umap-learn

In [ ]:
import keras
from keras import layers
from lxml import html
import numpy as np
import pandas as pd
import re
import requests
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from tqdm import tqdm
import umap

# Get MIND data

In [ ]:
behaviors = pd.read_csv("/content/drive/My Drive/Projet 16/Long Term/PyG/data/dev/behaviors.tsv",
                        sep="\t",
                        header=None,
                        names=["impression_id", "user_id", "time", "history", "impressions"])

behaviors.sample(frac=1, random_state=42).head()

,impression_id,user_id,time,history,impressions
181576,181577,U52930,11/15/2019 2:15:21 AM,N128643 N42646 N49511 N13423 N71728 N107705 N9...,N83707-0 N26122-0 N32993-0 N80770-0 N86609-0 N...
239312,239313,U561400,11/15/2019 8:03:52 AM,N128965 N61504 N13675 N123743 N68429 N94908 N1...,N30206-0 N29160-1 N58465-0 N122640-0 N54368-0 ...
247869,247870,U182031,11/15/2019 12:46:10 PM,N128736 N46591 N35797 N90185 N27258 N109816 N1...,N58760-0 N29160-0 N112536-0 N21018-0 N46555-1 ...
280282,280283,U655206,11/15/2019 6:10:23 AM,N4857 N85484 N49079 N27668 N38454 N24885 N1289...,N54368-0 N18190-1
290030,290031,U675991,11/15/2019 12:19:38 PM,N22660 N41709 N62019,N21018-0 N129416-0 N119999-0 N46641-0 N29160-0...


In [ ]:
news = pd.read_csv("/content/drive/My Drive/Projet 16/Long Term/PyG/data/dev/news.tsv",
                   sep="\t",
                   header=None,
                   names=["news_id", "category", "sub_category", "title", "abstract", "url", "title_entities", "abstract_entites"])

news.sample(frac=1, random_state=42).head()

,news_id,category,sub_category,title,abstract,url,title_entities,abstract_entites
39031,N44668,video,news,US Attorney John Durham now conducting a crimi...,Should officials who started the Russia probe ...,https://assets.msn.com/labs/mind/AAJnTiK.html,"[{""Label"": ""Special Counsel investigation (201...","[{""Label"": ""Special Counsel investigation (201..."
64896,N54249,news,newsscienceandtechnology,Google CEO cuts back biweekly meetings after r...,The TGIF town halls have been rescheduled as m...,https://assets.msn.com/labs/mind/BBWPgel.html,[],"[{""Label"": ""TGIF (TV programming block)"", ""Typ..."
34835,N113757,autos,autosnews,Greenburgh: Man suffers serious injuries in cr...,Greenburgh detectives are still investigating ...,https://assets.msn.com/labs/mind/AAJiTMe.html,"[{""Label"": ""Greenburgh, New York"", ""Type"": ""G""...","[{""Label"": ""Greenburgh, New York"", ""Type"": ""G""..."
54782,N31350,travel,travelarticle,"'Joker' Fans Come For The Stairs, Locals Hope ...",People who live and work at the real-life High...,https://assets.msn.com/labs/mind/AAJ97FU.html,[],"[{""Label"": ""Highbridge, Bronx"", ""Type"": ""G"", ""..."
16435,N91140,sports,football_nfl,The Linc - Jordan Howard says he'd definitely ...,Philadelphia Eagles news and links for 11/3/19.,https://assets.msn.com/labs/mind/AAJLDsP.html,"[{""Label"": ""Jordan Howard"", ""Type"": ""P"", ""Wiki...","[{""Label"": ""Philadelphia Eagles"", ""Type"": ""O"",..."


# Building dataset

## For this model, the objective is to retrieve relevant features of the article to capture a trend in the recent clicked news of a user. To do so, the following web scraping script is quering the URL associated with a news article and gets the entire text in a final string. The conditions to have a relevant dataset is to have enough links redirecting to actual text that can be analyzed.

In [ ]:
list_history = []
list_candidate = []
compt = 0

def data_loader(idx, user_id):
    temp_history = []
    temp_candidate = []
    candidate_is_valid = False
    history_is_valid = False
    if type(behaviors.loc[behaviors['user_id'] == user_id]['history'].values[0]) == str and type(behaviors.loc[behaviors['user_id'] == user_id]['impressions'].values[0]) == str:
        global compt
        compt += 1
        history_split = behaviors.loc[behaviors['user_id'] == user_id]['history'].values[0].split(' ')
        candidate_split = behaviors.loc[behaviors['user_id'] == user_id]['impressions'].values[0].split(' ')
        for i, item in enumerate(candidate_split):
            if (news.loc[news['news_id'] == item[:-2]]['url'].values != '' or news.loc[news['news_id'] == item[:-2]]['url'].values.size > 0) and int(item[-1]) == 1:
                temp_candidate.append(item[:-2])
                candidate_is_valid = True
                break
        if len(history_split) > 8:
            user_history = []
            for item in history_split:
                if (news.loc[news['news_id'] == item[:-2]]['url'].values != '' or news.loc[news['news_id'] == item[:-2]]['url'].values.size > 0) and (len(user_history) < 5) and candidate_is_valid:
                    user_history.append(item)
                    history_is_valid = True
            if history_is_valid and candidate_is_valid:
                for it in temp_candidate:
                    list_candidate.append(it)
                list_history.append(user_history)

In [ ]:
def old_data_loader(idx, user_id):
    if type(behaviors.loc[behaviors['user_id'] == user_id]['history'].values[0]) == str and type(behaviors.loc[behaviors['user_id'] == user_id]['impressions'].values[0]) == str:
        global compt
        compt += 1
        history_split = behaviors.loc[behaviors['user_id'] == user_id]['history'].values[0].split(' ')
        candidate_split = behaviors.loc[behaviors['user_id'] == user_id]['impressions'].values[0].split(' ')
        for i, item in enumerate(candidate_split):
            if (news.loc[news['news_id'] == item[:-2]]['url'].values != '' or news.loc[news['news_id'] == item[:-2]]['url'].values.size > 0) and int(item[-1]) == 1:
                list_candidate.append(item[:-2])
                break
        if len(history_split) > 8:
            user_history = []
            for item in history_split:
                if (news.loc[news['news_id'] == item[:-2]]['url'].values != '' or news.loc[news['news_id'] == item[:-2]]['url'].values.size > 0) and len(user_history) < 5:
                    user_history.append(item)
                list_history.append(user_history)

In [ ]:
for idx, user_id in enumerate(behaviors['user_id'][:100]):
    data_loader(idx, user_id)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


# Text encoding

## To obtain the features from the articles that are retrieved, the idea is to use a BERT-based model trained for this purpose. The best for this task is distilroBERTa. Once we have the model, it is possible to create the article embeddings used to match the input of recently clicked news to the next predicted news.

In [ ]:
encoder = SentenceTransformer('all-distilroberta-v1')
output.clear()

In [ ]:
def article_embedding(news_id):
    news_url = news.loc[news['news_id'] == news_id]['url'].values[0]
    page = requests.get(news_url)
    tree = html.fromstring(page.content)
    list_title = tree.xpath('//header//text()') + tree.xpath('//title//text()')
    list_text = tree.xpath('//p//text()')
    title = ''
    for element in list_title:
        element = element.replace('\r\n', '')
        if bool(re.match('(?=.*[a-zA-Z])', element)):
            title = element
    article = "title: " + title + "\nbody: "
    for element in list_text:
        article += element.replace("\xa0", " ")
    return encoder.encode(article), article

In [ ]:
for idx, item in enumerate(list_history):
    if len(item) < 5:
        list_history.pop(idx)

In [ ]:
x = []
umap_embedding = []

for user in tqdm(list_history):
    user_emb = []
    for news_id in user:
        emb = article_embedding(news_id)[0]
        user_emb.append(emb)
        umap_embedding.append(emb)
    x.append(user_emb)

# x_data = np.array(x, dtype=object).astype('float32')

100%|██████████| 74/74 [08:29<00:00,  6.88s/it]


In [ ]:
x_data = np.array(x, dtype=object).astype('float32')

In [ ]:
x_data.shape

(74, 5, 768)

In [ ]:
x_data[1, 2, :]

array([ 7.42005743e-03, -8.50299895e-02,  7.55199883e-03, -5.06412536e-02,
        4.37142141e-02,  8.44200179e-02, -3.28981727e-02, -1.00805186e-01,
        4.17859554e-02, -3.08148637e-02, -1.82543900e-02, -9.92955826e-03,
       -2.20263675e-02, -5.02972528e-02,  3.34116467e-03, -5.85550219e-02,
       -1.54054025e-02, -3.01610008e-02, -1.39096044e-02, -2.95141689e-03,
        5.99940717e-02,  2.01475015e-03, -4.37411740e-02,  2.81814709e-02,
       -7.19337433e-04, -3.28800082e-02,  1.02490000e-02, -1.42163597e-02,
       -2.77688913e-02,  7.49920867e-03, -7.89834280e-03,  2.20843614e-03,
       -3.81009206e-02,  1.87821873e-02, -4.34420370e-02, -1.58622849e-03,
        2.29047071e-02, -3.23821194e-02,  2.19417810e-02,  4.07310538e-02,
       -3.51853408e-02, -5.34468470e-03,  4.19711461e-03,  1.35367103e-02,
        5.09892404e-02, -5.01499325e-03, -5.20759672e-02, -4.70218435e-02,
       -1.17509617e-02, -4.79227379e-02,  2.11690813e-02, -2.15778966e-02,
        1.60087198e-02, -

# Dimension reduction

## Because the final vectors have high dimensions, we chose to use dimension reduction algorithms that could help the model converge and more correctly predict the next vector. The algorithm used is UMAP and is one of the best for this task.

In [ ]:
umap_emb = umap.UMAP(n_neighbors=15, 
                     n_components=69, 
                     metric='cosine').fit_transform(umap_embedding)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
umap_emb.shape

(370, 69)

In [ ]:
umap_emb[1]

array([5.05782  , 0.7971749, 1.7698562, 2.1316729, 5.065205 , 5.2560177,
       1.7082068, 7.2333183, 7.2903867, 4.255931 , 5.598235 , 4.735095 ,
       6.1532655, 3.1924815, 4.2894034, 4.4589076, 4.6292834, 6.171775 ,
       4.073825 , 6.0064993, 4.0732226, 4.667424 , 5.6596246, 2.5283165,
       5.717544 , 7.531137 , 5.5299597, 4.9989357, 4.2748513, 5.5764713,
       5.085316 , 4.4749246, 5.583394 , 4.622784 , 5.5717974, 4.2220607,
       4.538298 , 4.5998282, 5.567291 , 5.315018 , 5.5774636, 5.546187 ,
       5.0962157, 4.5654883, 4.955469 , 5.1977882, 5.306213 , 5.1386213,
       4.723305 , 4.4464006, 6.0125637, 4.5090747, 4.428159 , 3.7792788,
       5.1179056, 5.54784  , 5.8408175, 5.5282054, 5.5833545, 5.6447887,
       5.892504 , 5.1190777, 6.19496  , 4.4239974, 4.0358906, 4.6147733,
       6.0577617, 4.7605066, 6.447514 ], dtype=float32)

In [ ]:
x_umap = []

for i in range(len(umap_emb)//5):
    x_umap.append(umap_emb[i*5:i*5+5])

In [ ]:
x_umap = np.array(x_umap, dtype=object).astype('float32')
x_umap.shape

(74, 5, 69)

In [ ]:
x_umap[1, 2, :]

array([4.5779204, 0.7612202, 1.8815739, 2.249155 , 5.4481487, 5.076612 ,
       1.991798 , 7.295516 , 7.0167327, 3.736155 , 5.417722 , 4.5661516,
       6.4154515, 2.9691656, 4.1425867, 4.327845 , 4.9765434, 6.2208276,
       4.4805713, 6.680025 , 4.2834835, 4.7473435, 5.8370905, 2.9348223,
       5.854182 , 7.6328697, 5.903416 , 4.7009034, 4.247373 , 5.379615 ,
       5.4537544, 4.1266437, 5.766979 , 4.632747 , 5.82228  , 4.7836   ,
       5.152286 , 4.669774 , 5.5662956, 5.0485263, 5.930966 , 5.7533636,
       5.8415785, 4.785646 , 4.8089314, 5.3907533, 4.8018165, 5.1637306,
       4.218942 , 4.3460884, 5.9497895, 4.8955135, 4.716818 , 4.2566857,
       4.704751 , 5.094411 , 5.7287884, 4.9907994, 5.447152 , 5.320242 ,
       5.8531094, 5.1673427, 6.317832 , 4.221317 , 4.121166 , 4.7499485,
       6.6026697, 4.697438 , 6.6407423], dtype=float32)

In [ ]:
np.save('/content/drive/My Drive/Projet 16/Short Term/history_embedding.npy', x_data)

In [ ]:
y = []

for news_id in tqdm(list_candidate[:74]):
    y.append(article_embedding(news_id)[0])

# y_label = np.array(y, dtype=object)

100%|██████████| 74/74 [01:24<00:00,  1.14s/it]


In [ ]:
y_label = np.array(y, dtype=object).astype('float32')

In [ ]:
y_label.shape

(74, 768)

In [ ]:
y_umap = umap.UMAP(n_neighbors=2, 
                   n_components=69, 
                   metric='cosine').fit_transform(y_label)

In [ ]:
y_umap.shape

(74, 69)

In [ ]:
np.save('/content/drive/My Drive/Projet 16/Short Term/candidate_embedding.npy', y_label)

# Model

## The model is a simple GRU layer that will go through the article embeddings and try to predict the next one.

In [ ]:
# Input shape using the last 5 clicked news with 768 features for embedding but dimension reduced to 256
# Dimension of the output feature space of 128 as stater
# Dense layer to obtain a candidate embedding of shape (1, 1, 256)

inputs = layers.Input(shape=(5, 256))
gru = layers.GRU(128, return_sequences=True, return_state=True)
dense = layers.Dense(256, activation='sigmoid')

# _, h_0   = gru(inputs=tf.expand_dims(inputs[:, 0], axis=1), initial_state=None)
# _, h_1   = gru(inputs=tf.expand_dims(inputs[:, 1], axis=1), initial_state=h_0)
# _, h_2   = gru(inputs=tf.expand_dims(inputs[:, 2], axis=1), initial_state=h_1)
# _, h_3   = gru(inputs=tf.expand_dims(inputs[:, 3], axis=1), initial_state=h_2)
# out, h_4 = gru(inputs=tf.expand_dims(inputs[:, 4], axis=1), initial_state=h_3)

out, _ = gru(inputs=tf.expand_dims(inputs, axis=1), initial_state=None)

short_emb = dense(out)

short_interest = keras.Model(inputs, short_emb)

short_interest.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

print(short_interest.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 69)]      0           []                               
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 69)          0           ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None, 69)          0           ['input_1[0][0]']                
 icingOpLambda)                                                                                   
                                                                                              

In [ ]:
# Callbacks to save the best model on the accuracy
checkpoint_filepath = '/content/drive/My Drive/Projet 16/Encoder/models/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                            monitor='val_accuracy',
                                                            mode='max',
                                                            save_best_only=True)

In [ ]:
# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

# Evaluating

## In the following example, we can see the model has not learnt from the data in entry. The leads to improve the results are to reduce the dimension using an autoencoder that will ensure the encoding gives the exact input if decoded. Moreover, it is possible the simple GRU is not enough to analyze the trend because there is not enough data for one user. The idea will be to add more recent clicked news for each user and give a larger sequence of vectors for the GRU model to learn from it

In [ ]:
short_interest.fit(x_umap, tf.expand_dims(y_umap, axis=1), validation_split=0.05, epochs=10, batch_size=2) # callbacks=[model_checkpoint_callback])

Epoch 1/10
35/35 [==============================] - 3s 67ms/step - loss: 4.2531 - accuracy: 0.0000e+00 - val_loss: 4.0679 - val_accuracy: 0.0000e+00
Epoch 2/10
35/35 [==============================] - 2s 63ms/step - loss: 4.1073 - accuracy: 0.0000e+00 - val_loss: 4.0425 - val_accuracy: 0.0000e+00
Epoch 3/10
35/35 [==============================] - 2s 63ms/step - loss: 4.0968 - accuracy: 0.0000e+00 - val_loss: 4.0381 - val_accuracy: 0.0000e+00
Epoch 4/10
35/35 [==============================] - 2s 61ms/step - loss: 4.0941 - accuracy: 0.0000e+00 - val_loss: 4.0364 - val_accuracy: 0.0000e+00
Epoch 5/10
35/35 [==============================] - 2s 62ms/step - loss: 4.0930 - accuracy: 0.0000e+00 - val_loss: 4.0356 - val_accuracy: 0.0000e+00
Epoch 6/10
35/35 [==============================] - 2s 62ms/step - loss: 4.0924 - accuracy: 0.0000e+00 - val_loss: 4.0352 - val_accuracy: 0.0000e+00
Epoch 7/10
35/35 [==============================] - 2s 64ms/step - loss: 4.0920 - accuracy: 0.0000e+00 - v